<a href="https://colab.research.google.com/github/eric-castillo05/emotions-wheel-nlp/blob/main/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [30]:
import polars as pl
from datasets import load_dataset
import pickle

In [2]:
ds = load_dataset("Jsevisal/go_emotions_wheel")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
df = pl.from_dicts(ds['train'][:])

In [4]:
df.head()

text,labels,id
str,list[i64],str
"""My favourite food is anything …",[5],"""eebbqej"""
"""Now if he does off himself, ev…",[5],"""ed00q6i"""
"""WHY THE FUCK IS BAYLESS ISOING""",[7],"""eezlygj"""
"""To make her feel threatened""",[2],"""ed7ypvh"""
"""Dirty Southern Wankers""",[7],"""ed0bdzj"""


In [5]:
df.sample(10)

text,labels,id
str,list[i64],str
"""Chasing thoughts and storing t…",[1],"""ed6eaf9"""
"""That boy ain’t right""",[7],"""ed7f2g9"""
"""That's what we're doing. It's …",[5],"""edw3uza"""
"""Well to be fair to these idiot…",[7],"""eei3vrb"""
"""They're not going to read thei…",[7],"""eegdb9r"""
"""Well it looks like I am going …",[3],"""edu13rb"""
"""Looks like modern art tbh but …",[0],"""eduk4z3"""
"""They will just throw themselve…",[5],"""ef4hcdo"""
"""Sour Tangie!""",[5],"""edhwi8w"""


In [6]:
import spacy
import spacy_cleaner
from spacy_cleaner.processing import removers, replacers, mutators

In [7]:
!python3 -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
nlp = spacy.load("en_core_web_lg")

In [10]:
docs[:10]

["My favourite food is anything I didn't have to cook myself.",
 'Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead',
 'WHY THE FUCK IS BAYLESS ISOING',
 'To make her feel threatened',
 'Dirty Southern Wankers',
 "OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe PlAyOfFs! Dumbass Broncos fans circa December 2015.",
 'Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝',
 'We need more boards and to create a bit more space for [NAME]. Then we’ll be good.',
 'Damn youtube and outrage drama is super lucrative for reddit',
 'It might be linked to the trust factor of your friend.']

In [11]:
from spacy_cleaner import processing, Cleaner

pipeline = spacy_cleaner.Cleaner(
    nlp,
    processing.remove_number_token,
    processing.remove_punctuation_token,
    processing.mutate_lemma_token,
)

In [12]:
def clean(text, nlp):
    doc = nlp(text.lower())
    tokens = []
    for token in doc:
        if not token.is_digit and not token.is_punct and not token.is_space:
            tokens.append(token.lemma_)
    return ' '.join(tokens)

docs = df.select('text').to_series().to_list()
r = [clean(doc, nlp) for doc in docs]

In [13]:
df = df.with_columns(
    pl.Series('clean-text', r).alias('clean-text')
)

In [14]:
df.head()

text,labels,id,clean-text
str,list[i64],str,str
"""My favourite food is anything …",[5],"""eebbqej""","""my favourite food be anything …"
"""Now if he does off himself, ev…",[5],"""ed00q6i""","""now if he do off himself every…"
"""WHY THE FUCK IS BAYLESS ISOING""",[7],"""eezlygj""","""why the fuck be bayless isoing"""
"""To make her feel threatened""",[2],"""ed7ypvh""","""to make she feel threaten"""
"""Dirty Southern Wankers""",[7],"""ed0bdzj""","""dirty southern wanker"""


In [31]:
with open('train-clean.pkl', 'wb') as f:
  pickle.dump(df, f)

In [18]:
with open('train-clean.pkl', 'rb') as f:
  df_loaded = pickle.load(f)

In [23]:
df_test = pl.from_dicts(ds['test'][:])

In [25]:
def clean(text, nlp):
    doc = nlp(text.lower())
    tokens = []
    for token in doc:
        if not token.is_digit and not token.is_punct and not token.is_space:
            tokens.append(token.lemma_)
    return ' '.join(tokens)

docs = df_test.select('text').to_series().to_list()
r = [clean(doc, nlp) for doc in docs]

In [26]:
df_test = df_test.with_columns(
    pl.Series('clean-text', r).alias('clean-text')
)

In [27]:
df_test.head()

text,labels,id,clean-text
str,list[i64],str,str
"""I’m really sorry about your si…",[4],"""eecwqtt""","""I ’m really sorry about your s…"
"""It's wonderful because it's aw…",[0],"""ed5f85d""","""it be wonderful because it be …"
"""Kings fan here, good luck to y…",[0],"""een27c3""","""kings fan here good luck to yo…"
"""I didn't know that, thank you …",[0],"""eelgwd1""","""I do not know that thank you f…"
"""They got bored from haunting e…",[5],"""eem5uti""","""they get bored from haunt eart…"


In [32]:
with open('test-clean.pkl', 'wb') as f:
  pickle.dump(df, f)

In [35]:
import tensorflow_hub as hub
import tensorflow as tf

In [39]:
model = "https://tfhub.dev/google/elmo/3"
elmo = hub.load(model)

In [43]:
embeddings = elmo.signatures['default'](x)

TypeError: Binding inputs to tf.function failed due to `too many positional arguments`. Received args: (['Roasted ants are a popular snack in Columbia'],) and kwargs: {} for signature: () -> Dict[['lstm_outputs1', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['lstm_outputs2', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['word_emb', TensorSpec(shape=(None, None, 512), dtype=tf.float32, name=None)], ['elmo', TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None)], ['sequence_len', TensorSpec(shape=(None,), dtype=tf.int32, name=None)], ['default', TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None)]].
Fallback to flat signature also failed due to: pruned(text): expected argument #0(zero-based) to be a Tensor; got list (['Roasted ants are a popular snack in Columbia']).